In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.realestate.com.au/rent/in-west+perth%2c+wa+6005%3b+floreat%2c+wa+6014%3b+leederville%2c+wa+6007%3b+cottesloe%2c+wa+6011%3b+north+perth%2c+wa+6006%3b/list-1?activeSort=price-desc"
r = requests.get(url)
c = r.content #download content of webpage
soup = BeautifulSoup(c, "html.parser")
data = soup.find_all("li",{"class":"search-results__listing"}) #get dataset of content needed
total_pages = int(soup.find_all("a",{"class":"paginator__page-button"})[-1].text) #find total number of pages in above search


df_list = [] #list to create dataframe from dictionaries

for page in range(1,total_pages):    
    r = requests.get(url+"&page="+str(page))
    c = r.content #download content of webpage

    soup = BeautifulSoup(c, "html.parser")
    data = soup.find_all("li",{"class":"search-results__listing"}) #get dataset of content needed

    for item in data:
        d = {} #dictionary to store iterations
        try:
            d['Price'] = item.find("p",{"class":"listing-result__price"}).text #price data
            d['Address'] = item.find("a" ,{"class":"address is-two-lines listing-result__address"}).text #address
            d['Beds'] = item.find_all("span" ,{"class":"property-feature__feature"})[0].text #number of beds
            d['Baths'] = item.find_all("span" ,{"class":"property-feature__feature"})[1].text #number of baths
            d['Parking'] = item.find_all("span" ,{"class":"property-feature__feature"})[2].text #number of parking bays
            d['Link'] = item.find("a", {"class":"address is-two-lines listing-result__address"}).get('href') #link address
        except:
            pass
        df_list.append(d)

IndexError: list index out of range

In [6]:
rentals = pd.DataFrame(df_list)

In [10]:
rentals.dropna().set_index('Address').to_csv('houselist.csv')